In [1]:
# draft_1
ROW = 0 # indexing purposes -> Python start the index from 0
COLUMN = 1 # indexing purpose

class SampleEnvironment():
    def __init__(self, n_rows=1, n_cols=5):
        self.time = 0
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.pedestrian_destination = self.create_pedestrian_destination()
        self.pedestrian = self.create_pedestrian()
        self.grid = self.create_grid(self.pedestrian, n_rows, n_cols)

    @staticmethod
    def create_pedestrian_destination(): # will later be randomized
        destination_row = 0
        destination_col = 4
        return (destination_row, destination_col)

    @staticmethod
    def create_pedestrian(): # will later be randomized and increased the quantity
        pedestrian_row = 0
        pedestrian_col = 0
        return (pedestrian_row, pedestrian_col)

    @staticmethod
    def create_grid(pedestrian, n_rows, n_cols):
        grid = []
        for idx_row in range(n_rows):
            column=  []
            for idx_column in range(n_cols):
                column.append(0)
            grid.append(column)
        grid[pedestrian[ROW]][pedestrian[COLUMN]] = 1
        return grid

    @staticmethod    
    def update_pedestrian_location(pedestrian, n_rows, n_cols): # will later have multiple ways to "walk"
        pedestrian_row = pedestrian[ROW]
        pedestrian_col = pedestrian[COLUMN] + 1
        return(pedestrian_row, pedestrian_col)

    def reset(self):
        self.time = 0
        self.pedestrian = self.create_pedestrian()
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.pedestrian = self.update_pedestrian_location(self.pedestrian, self.n_rows, self.n_cols)
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        newobs = self.grid
        if self.pedestrian == self.pedestrian_destination:
            done = True
        else:
            done = False
        info = {
            'goal' : self.pedestrian_destination,
            'time' : self.time
        }
        return newobs, done, info

    def render(self): # will later change into grid
        for rows in self.grid:
            print(rows)

In [2]:
env = SampleEnvironment()

In [3]:
obs = env.reset()
print("obs : {}".format(obs))

obs : [[1, 0, 0, 0, 0]]


In [4]:
env.render()

[1, 0, 0, 0, 0]


In [5]:
for i in range(4):
    newobs, done, info = env.step()
    print('newobs : {}'.format(newobs))
    print('done : {}'.format(done))
    print('info : {}'.format(info))
    i = i+1

newobs : [[0, 1, 0, 0, 0]]
done : False
info : {'goal': (0, 4), 'time': 1}
newobs : [[0, 0, 1, 0, 0]]
done : False
info : {'goal': (0, 4), 'time': 2}
newobs : [[0, 0, 0, 1, 0]]
done : False
info : {'goal': (0, 4), 'time': 3}
newobs : [[0, 0, 0, 0, 1]]
done : True
info : {'goal': (0, 4), 'time': 4}


In [1]:
from itertools import product

# draft_1 + street light
ROW = 0 # indexing purposes -> Python start the index from 0
COLUMN = 1 # indexing purpose

class SampleEnvironment():
    def __init__(self, n_rows=1, n_cols=5, coverage=2):
        self.time = 0
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.coverage = coverage
        self.pedestrian_destination = self.create_pedestrian_destination()
        self.pedestrian = self.create_pedestrian()
        self.grid = self.create_grid(self.pedestrian, n_rows, n_cols)
        self.street_light_loc = self.create_street_light()
        self.street_coverage = self.street_light_coverage()
        self.light_grid = self.create_light_grid(self.street_light_loc, n_rows, n_cols)
        self.light_brightness = self.street_light_brightness()

    @staticmethod
    def create_pedestrian_destination(): # will later be randomized
        destination_row = 0
        destination_col = 4
        return (destination_row, destination_col)

    @staticmethod
    def create_pedestrian(): # will later be randomized and increased the quantity
        pedestrian_row = 0
        pedestrian_col = 0
        return (pedestrian_row, pedestrian_col)

    @staticmethod
    def create_grid(pedestrian, n_rows, n_cols):
        grid = []
        for idx_row in range(n_rows):
            column=  []
            for idx_column in range(n_cols):
                column.append(0)
            grid.append(column)
        grid[pedestrian[ROW]][pedestrian[COLUMN]] = 1
        return grid

    @staticmethod    
    def update_pedestrian_location(pedestrian, n_rows, n_cols): # will later have multiple ways to "walk"
        pedestrian_row = pedestrian[ROW]
        pedestrian_col = pedestrian[COLUMN] + 1
        return(pedestrian_row, pedestrian_col)

    @staticmethod
    def create_street_light():
        sl_row = 0
        sl_col = 2
        return(sl_row, sl_col)

    @staticmethod
    def create_light_grid(light, n_rows, n_cols):
        light_grid = []
        for idx_row in range(n_rows):
            column=  []
            for idx_column in range(n_cols):
                column.append(0)
            light_grid.append(column)
        light_grid[light[ROW]][light[COLUMN]] = 1
        return light_grid


    def street_light_coverage(self):
        coverage_row_from = self.street_light_loc[ROW] - self.coverage
        coverage_row_to = self.street_light_loc[ROW] + self.coverage
        coverage_col_from = self.street_light_loc[COLUMN] - self.coverage
        coverage_col_to = self.street_light_loc[COLUMN] + self.coverage

        if coverage_row_from < (self.n_rows - 1) or coverage_row_to > (self.n_rows - 1):
            coverage_row_from = [self.street_light_loc[ROW]]
            coverage_row_to = [self.street_light_loc[ROW]]
        else:
            coverage_row_from
            coverage_row_to = coverage_row_to + 1

        if coverage_col_from < 0 or coverage_col_to > (self.n_cols - 1):
            coverage_col_from = [self.street_light_loc[COLUMN]]
            coverage_col_to = [self.street_light_loc[COLUMN]]
        else:
            coverage_col_from
            coverage_col_to = coverage_col_to + 1

        if coverage_row_from == coverage_row_to:
            coverage_row = coverage_row_from
        else:
            coverage_row = list(range(coverage_row_from, coverage_row_to))

        if coverage_col_from == coverage_col_to:
            coverage_col = coverage_col_from
        else:
            coverage_col = list(range(coverage_col_from, coverage_col_to))
        
        coverage_coor = []
        for row, column in product(coverage_row, coverage_col):
            coverage_coor.append((row, column))

        return coverage_coor

    def street_light_brightness(self):
        test_coor = self.street_coverage.count((self.pedestrian[ROW], self.pedestrian[COLUMN]))
        if test_coor == 0:
            brightness = 0
        else:
            brightness = 3
        return brightness

    def reset(self):
        self.time = 0
        self.pedestrian = self.create_pedestrian()
        self.pedestrian_destination = self.create_pedestrian_destination()
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        return self.grid

    def step(self):
        self.time = self.time + 1
        self.pedestrian = self.update_pedestrian_location(self.pedestrian, self.n_rows, self.n_cols)
        self.grid = self.create_grid(self.pedestrian, self.n_rows, self.n_cols)
        newobs = self.grid
        if self.pedestrian == self.pedestrian_destination:
            done = True
        else:
            done = False
        info = {
            'goal' : self.pedestrian_destination,
            'time' : self.time
        }
        return newobs, done, info

    def render(self): # will later change into grid
        for rows in self.grid:
            print(rows)

a = SampleEnvironment()
print(a.street_coverage)
print(a.light_brightness)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
3
